# Sklearn FeatureUnion
- Use custom transformers for feature engineering
- Then merge the features horizontally for feeding into an ML classifier

## FeatureUnion & Pipelines with Pandas
https://towardsdatascience.com/custom-transformers-and-ml-data-pipelines-with-python-20ea2a7adb65

In [68]:
import numpy as np 
import pandas as pd
import spacy
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
spacy.prefer_gpu()

True

In [3]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.matcher import Matcher
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [4]:
# !python -m spacy download en_core_web_sm

# nlp = spacy.load("en_core_web_sm")
nlp = English()

In [5]:
# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

In [6]:
# Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)

In [7]:
# Create matcher for hashtags
matcher = Matcher(nlp.vocab)
matcher.add('HASHTAG', None, [{'ORTH': '#'}, {'IS_ASCII': True}])

# Feature selector transformer
- Feed it the columns you want, and it returns a dataframe with just those features

In [8]:
# Custom Transformer that extracts columns passed as argument to its constructor 
class FeatureSelector( BaseEstimator, TransformerMixin ):
    # Class Constructor 
    def __init__(self, feature_names):
        self.feature_names = feature_names 
    
    # Return self nothing else to do here    
    def fit(self, X, y = None):
        return self 
    
    # Method that describes what we need this transformer to do
    # This one pulls up the list of feature columns you pass in and returns just those columns
    def transform(self, X, y = None):
        return X[self.feature_names] 

# Text processing transformer
- Take in tweet text
- Create features
    - contains hashtag
    - isupper
    - islower
    - has mispellings

# Create the text pipeline

In [205]:
# Custom transformer that takes in a string and returns new categorical features
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes in a list of values as its argument
    def __init__(self):
        pass
        
        
    # Return self nothing else to do here
    def fit(self, X, y = None):
        return self
    
    
    # Test helper func to just return the text in all lower case
    def is_lower(self, obj):
        if obj.islower():
            return 1
        else:
            return 0
    
    
    def is_upper(self, obj):
        if obj.isupper():
            return 1
        else:
            return 0


    # Transformer method to take in strings from a dataframe and return some extra features
    def transform(self, X , y = None):
        # Copy the incoming df to prevent setting on copy errors
        X = X.copy()
        
        # Return binary indicator of whether tweet is all lowercase
        X['is_lower'] = X['text'].apply(self.is_lower)
        
        # Return binary indicator of whether tweet is all uppercase
        X['is_upper'] = X['text'].apply(self.is_upper)
    
        # Drop original text col
        # The only thing remaining now will be the lowercased text
        X = X.drop('text', axis=1)
        
        # returns numpy array
        return X.values 
    
    # Transformer method to take in strings from a dataframe and return some extra features
    def fit_transform(self, X , y = None):
        # Copy the incoming df to prevent setting on copy errors
        X = X.copy()
        
        # Return binary indicator of whether tweet is all lowercase
        X['is_lower'] = X['text'].apply(self.is_lower)
        
        # Return binary indicator of whether tweet is all uppercase
        X['is_upper'] = X['text'].apply(self.is_upper)
    
        # Drop original text col
        # The only thing remaining now will be the lowercased text
        X = X.drop('text', axis=1)
        
        # returns numpy array
        return X.values 

In [235]:
# Custom transformer that tokenizes text
class TextTokenizerTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes in a list of values as its argument
    def __init__(self):
        pass
        
        
    # Return self nothing else to do here
    def fit(self, X, y = None):
        return self
    
    
    def spacy_tokenizer(self, obj):
        doc = nlp(obj)

        # Looks for hashtags
        matches = matcher(doc)
        spans = []
        for match_id, start, end in matches:
            spans.append(doc[start:end])

        for span in spans:
            span.merge()

        return [t.text.lower() for t in doc if t not in stop_words and not t.is_punct | t.is_space]


    # Transformer method to take in strings from a dataframe and return some extra features
    def transform(self, X , y = None):
        # Copy the incoming df to prevent setting on copy errors
        X = X.copy()
        
        X['tokens'] = X['text'].apply(self.spacy_tokenizer)
        
        X['tokens'] = " ".join(X['tokens'])
        return X['tokens']
    
    # Transformer method to take in strings from a dataframe and return some extra features
    def fit_transform(self, X , y = None):
        # Copy the incoming df to prevent setting on copy errors
        X = X.copy()
        
        X['tokens'] = X['text'].apply(self.spacy_tokenizer)
        
        X['tokens'] = " ".join(X['tokens'])
        return X['tokens']

In [207]:
# Custom transformer that takes in a string and returns some features
class TextTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes in a list of values as its argument
    def __init__(self):
        pass
        
    # Return self nothing else to do here
    def fit(self, X, y = None):
        return self
    
    
    def spacy_tokenizer(self, obj):
        doc = nlp(obj)

        # Looks for hashtags
        matches = matcher(doc)
        spans = []
        for match_id, start, end in matches:
            spans.append(doc[start:end])

        for span in spans:
            span.merge()

        return [t.text.lower() for t in doc if t not in stop_words and not t.is_punct | t.is_space]


    # Transformer method to take in strings from a dataframe and return some extra features
    def transform(self, X , y = None):
        # Copy the incoming df to prevent setting on copy errors
        X = X.copy()
        
        # Embed text as a bag of words using tfidf
        tfidf = TfidfVectorizer(tokenizer = self.spacy_tokenizer)
        X = tfidf.fit_transform(X['text'])
        
        # returns numpy array
        return X

In [268]:
class DenseTfidfVectorizer(TfidfVectorizer):
    def __init__(self):
        self.tfidf_model = TfidfVectorizer(tokenizer=self.spacy_tokenizer)
        
    def spacy_tokenizer(self, obj):
        doc = nlp(obj)

        # Looks for hashtags
        matches = matcher(doc)
        spans = []
        for match_id, start, end in matches:
            spans.append(doc[start:end])

        for span in spans:
            span.merge()

        return [t.text.lower() for t in doc if t not in stop_words and not t.is_punct | t.is_space]
        
    def transform(self, raw_documents, copy=True):
#         X = super().transform(raw_documents, copy=copy)
        X = self.tfidf_model.transform(raw_documents['text'], copy=copy)
#         df = pd.DataFrame(X.toarray(), columns=self.get_feature_names())
#         return df
#         return X.toarray()
        return X

    
    def fit_transform(self, raw_documents, y=None):
#         X = super().fit_transform(raw_documents, y=y)
        X = self.tfidf_model.fit_transform(raw_documents['text'], y=y)
#         df = pd.DataFrame(X.toarray(), columns=self.get_feature_names())
#         return df
#         return X.toarray()
        return X

In [269]:
test = DenseTfidfVectorizer()

In [271]:
test.fit_transform(X_train)

<7613x23223 sparse matrix of type '<class 'numpy.float64'>'
	with 109158 stored elements in Compressed Sparse Row format>

In [285]:
test.transform(X_test)

/home/alex/miniconda3/envs/spacy/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1897: FutureWarning: 'copy' param is unused and has been deprecated since version 0.22. Backward compatibility for 'copy' will be removed in 0.24.
  warnings.warn(msg, FutureWarning)


<50x23223 sparse matrix of type '<class 'numpy.float64'>'
	with 739 stored elements in Compressed Sparse Row format>

In [289]:
test_1 = FeatureSelector(['text']).transform(X_test)

In [291]:
test_2 = DenseTfidfVectorizer()

In [292]:
test_2.transform(test_1)

NotFittedError: The TF-IDF vectorizer is not fitted

In [264]:
# # Categrical features to pass down the categorical pipeline 
# categorical_features = ['date', 'waterfront', 'view', 'yr_renovated']

# #Numerical features to pass down the numerical pipeline 
# numerical_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
#                 'condition', 'grade', 'sqft_basement', 'yr_built']

# Categorical features (in this case it's still just 'text' until we include keywords and locations)
cat_features = ['text']

# Text features for text pipeline
text_features = ['text']

# #Defining the steps in the categorical pipeline 
# categorical_pipeline = Pipeline( steps = [ ( 'cat_selector', FeatureSelector(categorical_features) ),
                                  
#                                   ( 'cat_transformer', CategoricalTransformer() ), 
                                  
#                                   ( 'one_hot_encoder', OneHotEncoder( sparse = False ) ) ] )
    
# #Defining the steps in the numerical pipeline     
# numerical_pipeline = Pipeline( steps = [ ( 'num_selector', FeatureSelector(numerical_features) ),
                                  
#                                   ( 'num_transformer', NumericalTransformer() ),
                                  
#                                   ('imputer', SimpleImputer(strategy = 'median') ),
                                  
#                                   ( 'std_scaler', StandardScaler() ) ] )

# Define categorical pipeline
cat_pipeline = Pipeline(
    steps = [('cat_selector', FeatureSelector(cat_features)),
             ('cat_transformer', CategoricalTransformer()),
            ],
    verbose = True
)

# Define the text training pipeline
text_pipeline = Pipeline(
    steps = [('text_selector', FeatureSelector(text_features)),
#              ('text_transformer', TextTokenizerTransformer()),
             ('text_tfidf', DenseTfidfVectorizer())
            ],
    verbose = True
)

# Create the full pipeline

In [276]:
# #Combining numerical and categorical piepline into one full big pipeline horizontally 
# #using FeatureUnion
# full_pipeline = FeatureUnion( transformer_list = [ ( 'categorical_pipeline', categorical_pipeline ), 
                                                  
#                                                   ( 'numerical_pipeline', numerical_pipeline ) ] )

# Combine all our pipelines into a single one inside the FeatureUnion object
# Right now we only have one pipeline which is our text one
full_pipeline = FeatureUnion(
    transformer_list=[
                      ('text_pipeline', text_pipeline),
#                       ('cat_pipeline', cat_pipeline)
                     ]
)

# Test pipeline

In [198]:
train_df = pd.read_csv("data/train.csv")

In [199]:
train_small_df = train_df.sample(50, random_state=42)
train_small_df.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


# Can't run the full pipeline with the training and the model
- The data transformation part of the pipeline that does TFIDF will return different number of features based on the data fed in

# Solution: Separate the feature pipeline with the model pipeline

In [294]:
X_train = train_df.copy()
y_train = X_train.pop('target').values

In [296]:
text_pipeline.fit_transform(X_train)

[Pipeline] ..... (step 1 of 2) Processing text_selector, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing text_tfidf, total=   0.0s


<1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [295]:
# Process text and categorical features
X_train_processed = full_pipeline.fit_transform(X_train)

[Pipeline] ..... (step 1 of 2) Processing text_selector, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing text_tfidf, total=   0.0s


In [ ]:
lrcv =  LogisticRegressionCV(cv=5, random_state=42, n_jobs=-1)
lrcv.fit(X_train_processed, y_train)

In [172]:
# #The full pipeline as a step in another pipeline with an estimator as the final step
# full_pipeline_m = Pipeline(steps = [
#     ('full_pipeline', full_pipeline),
#     ('model', LogisticRegressionCV(cv=5, random_state=42, n_jobs=-1)) 
# ])

# #Can call fit on it just like any other pipeline
# full_pipeline_m.fit(X_train, y_train)

In [ ]:
X_test = train_df.copy().sample(50, random_state=42)
y_test = X_test.pop('target').values

In [278]:
# Preprocess test data
X_test_processed = full_pipeline.transform(X_test)

/home/alex/miniconda3/envs/spacy/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1897: FutureWarning: 'copy' param is unused and has been deprecated since version 0.22. Backward compatibility for 'copy' will be removed in 0.24.
  warnings.warn(msg, FutureWarning)


In [282]:
X_test_processed

<1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [280]:
X_train_processed

<1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [176]:
# Predict
y_pred = lrcv.predict(X_test_processed) 

ValueError: X has 544 features per sample; expecting 23225

In [ ]:
# full_pipeline_m.get_params()

In [ ]:
# %%time
# # Predicting with a test dataset
# predicted = pipe.predict(X_test)

# # Model Accuracy
# print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
# print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
# print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))